# Thử trước trên tập val

In [ ]:
Resnet_based_model = BuildModel(ModelName='ResNet152')
EfficientNet_based_model = BuildModel(ModelName='EfficientNetB7')
Resnet_based_model.load_weights('/content/drive/MyDrive/Models/ResNet152-16-0.0092.hdf5')
EfficientNet_based_model.load_weights('/content/drive/MyDrive/Models/EfficientNetB7-11-0.0150.hdf5')

In [ ]:
Resnet_val_pred = Resnet_based_model.predict(val_generator, steps=len(df_val_cut)//16)
EfficientNet_val_pred = EfficientNet_based_model.predict(val_generator, steps=len(df_val_cut)/16)

In [ ]:
val_y = df_val_cut['label'].astype(np.float32).values

Resnet_val_logloss = log_loss(val_y, Resnet_val_pred.flatten())
EfficientNet_val_logloss = log_loss(val_y, EfficientNet_val_pred.flatten())
print(f'Before Clipping,\nResnet_Logloss: {Resnet_val_logloss}\nEfficientNet_Logloss: {EfficientNet_val_logloss}')

# Blending trước + Clipping sau

In [ ]:
ensemble_pred_1 = 0.4*Resnet_val_pred + 0.6*EfficientNet_val_pred
print(f'Before Clipping, Blending_Logloss: {log_loss(val_y,ensemble_pred_1)}')

In [ ]:
clipped_ensemble_pred_1 = np.clip(ensemble_pred_1,0.04,0.96)
print(f'After Clipping, Blending_Logloss: {log_loss(val_y,clipped_ensemble_pred_1)}')

# Clipping trước + Blending sau

In [ ]:
Resnet_clipped_val_pred = np.clip(Resnet_val_pred.flatten(),0.04,0.96)
EfficientNet_clipped_val_pred = np.clip(EfficientNet_val_pred.flatten(),0.04,0.96)

Resnet_val_logloss = log_loss(val_y,Resnet_clipped_val_pred)
EfficientNet_val_logloss = log_loss(val_y,EfficientNet_clipped_val_pred)
print(f'After Clipping,\nResnet_Logloss: {Resnet_val_logloss}\nEfficientNet_Logloss: {EfficientNet_val_logloss}')

In [ ]:
ensemble_pred_2 = 0.5*Resnet_clipped_val_pred + 0.5*EfficientNet_clipped_val_pred
log_loss(val_y,ensemble_pred_2)

In [ ]:
# Ensemble

In [ ]:
#ResNet and EficientNet pred
import csv
Resnet_pred = []
EfficientNet_pred = []

with open(r'/content/drive/MyDrive/Models/AIIJC/submision.csv') as f1:
  csv_reader = csv.reader(f1)
  k = 0
  for row in csv_reader:
    if k == 0:
      pass
    if k != 0:
      Resnet_pred.append(float(row[1]))
      k = k+1
    k = k+1

with open(r'/content/drive/MyDrive/Models/AIIJC/submision (1).csv') as f2:
  csv_reader = csv.reader(f2)
  k = 0
  for row in csv_reader:
    if k == 0:
      pass
    if k != 0:
      EfficientNet_pred.append(float(row[1]))
    k = k+1

In [ ]:
Resnet_pred = np.array(Resnet_pred)
EfficientNet_pred = np.array(EfficientNet_pred)

In [ ]:
Resnet_clipped_pred = np.clip(Resnet_pred.flatten(),0.04,0.96)
EfficientNet_clipped_pred = np.clip(EfficientNet_pred.flatten(),0.04,0.96)

ensemble_pred = 0.5*Resnet_clipped_pred + 0.5*EfficientNet_clipped_pred

In [ ]:
ensemble_pred = 0.4*Resnet_pred + 0.6*EfficientNet_pred
clipped_ensemble_pred = np.clip(ensemble_pred,0.04,0.96)

In [ ]:
ensemble_pred.shape

In [ ]:
df_submit = pd.DataFrame({"name":df_test['name'],'pred':ensemble_pred.flatten()})
df_submit.to_csv("submision.csv",index = False)
df_submit.head(20)